In [6]:
%load_ext autoreload
%autoreload 2
%env ANYWIDGET_HMR=1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
env: ANYWIDGET_HMR=1


In [7]:
import numpy as np
import json
from pathlib import Path
data = json.loads(Path("/Users/mpiekenbrock/Downloads/miserables.json").read_text())

node_ids = np.unique([n["id"] for n in data["nodes"]])
src_ids = np.array([e["source"] for e in data["links"]])
tgt_ids = np.array([e["target"] for e in data["links"]])

src_ids = np.searchsorted(node_ids, src_ids)
tgt_ids = np.searchsorted(node_ids, tgt_ids)
node_ids = np.searchsorted(node_ids, node_ids)

edgelist = np.c_[src_ids, tgt_ids]

import traitlets
import anywidget
class Widget(anywidget.AnyWidget):
    _esm = "pixiplex/static/widget.js"
    _css = "pixiplex/static/widget.css"
    
    width = traitlets.Int(250).tag(sync=True)
    height = traitlets.Int(250).tag(sync=True)
    
    node_ids = traitlets.List([]).tag(sync=True)
    src_ids = traitlets.List([]).tag(sync=True)
    tgt_ids = traitlets.List([]).tag(sync=True)
    
    def __init__(self, node_ids: np.ndarray, edgelist: np.ndarray, width: int = 250, height: int = 250) -> None:
        super().__init__()
        self.width = width 
        self.height = height
        self.node_ids = list(node_ids)
        self.src_ids = list(edgelist[:,0])
        self.tgt_ids = list(edgelist[:,1])
    

widget = Widget(node_ids, edgelist, 400, 400)
widget

Widget(height=400, node_ids=[np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.…

In [3]:
widget.send({ "type" : "msg:init_graph"})

In [4]:
widget.send({ "type" : "msg:init_force"})

In [5]:
widget.send({ "type" : "msg:custom", "foo":"bar" })

In [ ]:
# %% Embed statically -- this works!
# import ipywidgets.embed
# ipywidgets.embed.embed_minimal_html("forcegraph_embed.html", views=[widget], drop_defaults=False)